## Connect to an online CosmosDB account which exists

For this section, we will explore and get hands-on with the .NET SDK for the Azure Cosmos DB SQL API.

After completing this module, you'll be able to:

- Integrate the Microsoft.Azure.Cosmos SDK library from NuGet
- Connect to an Azure Cosmos DB SQL API account using the SDK and .NET

***NuGet is the package manager for .NET. The NuGet client tools provide the ability to produce and consume packages. The NuGet Gallery is the central package repository used by all package authors and consumers.***

The **Microsoft.Azure.Cosmos** library is the latest version of the .NET SDK for Azure Cosmos DB SQL API.

The library is open-source and hosted online on **GitHub** at ***azure/azure-cosmos-dotnet-v3***. The opensource project conforms to the Microsoft Open Source Code of Conduct and accepts contributions and suggestions from the community.

The Microsoft.Azure.Cosmos library includes a namespace of the same name with common classes that you will explore later in this module including, but not limited to:

|Class|Description|
|---|---|
|**Microsoft.Azure.Cosmos.CosmosClient**|Client-side logical representation of an Azure Cosmos DB account and the primary class used for the SDK|
|**Microsoft.Azure.Cosmos.Database**|Logically represents a database client-side and includes common operations for database management|
|**Microsoft.Azure.Cosmos.Container**|Logically represents a container client-side and includes common operations for container management|

When working with the actual C# code, you will use a command in Powershell, similar to what we have in the below cell, in order to install the package in the .NET project folder (i.e. where you are developing the code).

In [ ]:
cd C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001
dotnet add package Microsoft.Azure.Cosmos
#dotnet add package Microsoft.Azure.Cosmos --version 3.22.1

  Determining projects to restore...
  Writing C:\Users\anna\AppData\Local\Temp\tmp18B6.tmp
info : Adding PackageReference for package 'Microsoft.Azure.Cosmos' into project 'C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001\app.csproj'.
info :   CACHE https://api.nuget.org/v3/registration5-gz-semver2/microsoft.azure.cosmos/index.json
info : Restoring packages for C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001\app.csproj...
info : Package 'Microsoft.Azure.Cosmos' is compatible with all the specified frameworks in project 'C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001\app.csproj'.
info : PackageReference for package 'Microsoft.Azure.Cosmos' version '3.26.1' updated in file 'C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001\app.csproj'.
info : Assets file has not changed. Skipping assets file writing. Path: C:\Users\anna\Documents\Courseware\DP420\NotebooksAnna\module_03_001\obj\project.assets.json
log  : R

In [ ]:
#Now let's connect to my Azure portal and fetch the Cosmos DB key
$subscriptionName = "annamicrosoft"
Connect-AzAccount -Subscription $subscriptionName


Account                  SubscriptionName TenantId                             Environment
-------                  ---------------- --------                             -----------
annavanyan@microsoft.com annamicrosoft    72f988bf-86f1-41af-91ab-2d7cd011db47 AzureCloud



In [ ]:

$resourceGroupName = "DP420LabArchiveCosmos"    # Resource Group must already exist
$accountName = "cosmosdbaccountanna001"         # Must be all lower case
$keyKind = "primary"                            # Other key kinds: secondary, primaryReadOnly, secondaryReadOnly

#The two below lines will print out my keys, so let's not do that for security reasons - but keeping it for info purposes
#Write-Host "List keys" 
#Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "Keys"

#The below variables are going to retrieve and store our connection string, primary key and endpoint of the Cosmos DB account.
#We will use them later on in the further C# cells to connect to the Cosmos DB account using the .NET SDK (i.e. C#)

$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName $resourceGroupName -Name $accountName -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName $resourceGroupName -Name $accountName).DocumentEndpoint

Now we need to install the packages that we will be using and transfer the variables that store the connection string, the primary master key and the endpoint into Cosmos DB C# code cells.
That is what we will do below : 

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos , 3.22.1"
#r "nuget: Bogus, 34.0.1"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

Installed Packages Bogus, 34.0.1 Microsoft.Azure.Cosmos, 3.22.1 Newtonsoft.Json, 13.0.1

Once the **Microsoft.Azure.Cosmos** library is imported, you can begin using the namespaces and classes within your .NET project.

### Import the namespace ###

Before using the library, you should import the **Microsoft.Azure.Cosmos** namespace using a `using` directive. The using directive allows you to use types within the namespace without being forced to fully qualify each type.

>`using System;`

>`using System.Linq;`

>`using Microsoft.Azure.Cosmos;`

In [ ]:
using System;
using System.Linq;
using Microsoft.Azure.Cosmos;

### Use the CosmosClient class

The two most common ways to create an instance for the **CosmosClient class** is to instantiate it with one of the following two constructors:

- A constructor that takes a single string value representing the connection string for the account.
- A constructor that takes two string values representing the endpoint and a key for the account.

This example uses a connection string in the AccountEndpoint=<account-endpoint>;AccountKey=<account-key> format with the fictional endpoint and key.

**Remember** - we retrieved the connection strinc from the Powershell cell earlier and passed the value in a variable called **connectionString**.



In [ ]:
CosmosClient client = new (connectionString);

Once the client instance is instantiated, you can use various methods directly. For example, you can asynchronously invoke the **ReadAccountAsync** method to get an object of type AccountProperties with various properties.

In [ ]:

AccountProperties account = await client.ReadAccountAsync();

So let us now print to the console the information about the Cosmos DB Account that we obtained from the cell above, by invoking the **ReadAccountAsync** method.

In [ ]:


Console.WriteLine($"Account Name:\t{account.Id}");
Console.WriteLine($"Primary Region:\t{account.WritableRegions.FirstOrDefault()?.Name}");

Account Name:	cosmosdbaccountanna001
Primary Region:	France Central


## Interact with a database

Once you have a client instance, you can retrieve or create a database using one of three methods:

- Retrieve an existing database using the name
- Create a new database passing in a unique database name
- Have the SDK check for the existence of the database and either create or retrieve it automatically

Any of these three methods will return an instance of type Database that you can use to interact with the database.

Let us first fetch an existing database in CosmosDB account using the below syntax : 

>`Database database = client.GetDatabase("theNameOfMyDatabase");`

In [ ]:
string myDBName = "CustomerProfile";
Database database = client.GetDatabase(myDBName);

We can create a database using one of the below methods : 

>`Database database = await client.CreateDatabaseAsync("cosmicworks");`

>`Database database = await client.CreateDatabaseIfNotExistsAsync("cosmicworks");`

Let us use the second method to create a database in Cosmos DB Account, names ***cosmicworks***

In [ ]:
string myDBToCreateName = "cosmicworks";
Database database = await client.CreateDatabaseIfNotExistsAsync(myDBToCreateName);

In order to specify the database throughput and depending on the .NET SDK version, we can use the options below : 

>|.NET SDK v2|.NET SDK v3|
>|---|---|
>|//set the throughput for the database <br> `RequestOptions options = new RequestOptions` <br> `{` <br> `OfferThroughput = 500` <br> `};` <br> <br> //create the database <br> `await client.CreateDatabaseIfNotExistsAsync(` <br> `new Database {Id = databaseName},` <br> <br> `options);` <br>|//create the database with throughput <br> `string databaseName = "MyDatabaseName";` <br> <br> `await this.cosmosClient.CreateDatabaseIfNotExistsAsync(` <br> <br> `id: databaseName,` <br> `throughput: 1000);`|


Now let us create a container within our newly created ***cosmicworks*** database.

In [ ]:
Container container = await database.CreateContainerIfNotExistsAsync(
    "products", 
    "/categoryId", 
    400
);

As you will notice, we specified the throughput in the code. In case if we would like to modify it in future, we can use the syntax below : 

In [ ]:
Container container = database.GetContainer("products");

ThroughputResponse throughput = await container.ReplaceThroughputAsync(
    ThroughputProperties.CreateManualThroughput(500));

In order to see the changes that we made, you can go to the Azure Portal -> Cosmos DB account -> the database ***cosmicworks*** -> **Scale and Settings** pane.

If you had already the page opened, please refresh the page completely to see the changes.

## Azure Cosmos DB Connectivity modes ##

How a client connects to Azure Cosmos DB has important performance implications, especially for observed client-side latency. Azure Cosmos DB offers a simple, open RESTful programming model over HTTPS called gateway mode. Additionally, it offers an efficient TCP protocol, which is also RESTful in its communication model and uses TLS for initial authentication and encrypting traffic, called direct mode.

### Available connectivity modes ###

The two available connectivity modes are:

- ##### Gateway mode #####

Gateway mode is supported on all SDK platforms. If your application runs within a corporate network with strict firewall restrictions, gateway mode is the best choice because it uses the standard HTTPS port and a single DNS endpoint. The performance tradeoff, however, is that gateway mode involves an additional network hop every time data is read from or written to Azure Cosmos DB. We also recommend gateway connection mode when you run applications in environments that have a limited number of socket connections.

When you use the SDK in Azure Functions, particularly in the Consumption plan, be aware of the current limits on connections.

- ##### Direct mode #####

Direct mode supports connectivity through TCP protocol and offers better performance because there are fewer network hops. The application connects directly to the backend replicas. Direct mode is currently only supported on .NET and Java SDK platforms.
